# Получение структурированного вывода

Получение данных, сгенерированных LLM, в виде структур формата JSON значительно упрощает их хранение и обработку.

И хотя модели GigaChat не поддерживают работу в режиме принудительной генерации JSON (т.н. *JSON mode*), существуют способы получить от них данные в структурированном формате.

В этом разделе вы узнаете как получить структурированный вывод от моделей GigaChat с помощью LangChain-метода `.with_structured_output()` или парсеров.

> ⚠️ Обратите внимание! На текущий момент (апрель 2025) GigaChat не поддерживает работу со схемами содержащими типы anyOf, oneOf, allOf, а также pydantic объектами, которые используют типы, приводящие к появлению таких конструкций в схемах, например Union. Пожалуйста избегайте таких типов данных при работе с GigaChat.

## Подготовка к работе

Для работы примеров установите необходимые зависимости и инициализируйте GigaChat.

### Установка зависимостей

Установите библиотеку langchain-gigachat:

### Инициализация GigaChat

Для работы с моделями GigaChat, нужно передать ключ авторизации GigaChat API.

Ключ можно указать в переменной среды `GIGACHAT_CREDENTIALS`, заданной в файле `.env` или созданной с помощью команды:

```sh
export GIGACHAT_CREDENTIALS=ключ_авториазации
```

При инициализации проверяется наличие переменной среды `GIGACHAT_CREDENTIALS` с заданным ключом авторизации GigaChat API.
Если переменная отсутствует, вы сможете указать ключ в поле **Введите ключ авторизации GigaChat API**.

О способах авторизации и поддерживаемых переменных среды — в [README библиотеки gigachat](https://github.com/ai-forever/gigachat).

In [ ]:
import getpass
import os
from dotenv import find_dotenv, load_dotenv

from langchain_gigachat.chat_models.gigachat import GigaChat

load_dotenv(find_dotenv())

if "GIGACHAT_CREDENTIALS" not in os.environ:
    os.environ["GIGACHAT_CREDENTIALS"] = getpass.getpass("Введите ключ авторизации GigaChat API: ")

model = GigaChat(
    model="GigaChat-2-Max",
    verify_ssl_certs=False,
)

## Использование `.with_structured_output()`

Метод `.with_structured_output()` принимает на вход схему с названиями полей, типами и описанием данных, которые должна вернуть модель.
Схема данных может быть описана в виде Pydantic- и TypedDict-классов, или JSON Schema.

### Описание вывода с помощью Pydantic-класса

Модели GigaChat могут возвращать Pydantic-объекты.
Для этого при вызове метода нужно передать схему данных в виде Pydantic-класса.

Преимущество использования Pydantic-классов в том, что Pydantic автоматически провалидирует сгенерированный вывод.
Так, если в ответе модели отсутствуют обязательные поля и указаны неверные типы данных, Pydantic вернет ошибку.

При описании структуры с помощью Pydantic-класса важно дать понятное название и подробное описание как самому классу, так и его аттрибутам.
Все эти данные будут использованы в промпте при отправке запроса в модель.

In [9]:
from typing import Optional

from pydantic import BaseModel, Field


# Pydantic 
class Book(BaseModel):
    """Книга, которая нужна пользователю"""
    title: str = Field(description="Название книги")
    author: Optional[str] = Field(
        default="неизвестен", description="Автор книги"
    )
    language: Optional[str] = Field(
        default="любой", description="Язык написания книги"
    )


structured_llm = model.with_structured_output(Book)

structured_llm.invoke("Найди '1984' Оруэлла в английском оригинале")

Book(title='1984', author='George Orwell', language='English')

> ⚠️ Для повышения качества можно также добавить в промпт автоматически сгенерированную инструкцию, которая поможет модели с более точной генерацией JSON. На текущий момент времени (апрель 2025) данный метод является наиболее эффективным с точки зрения качества работы со сложными данными.

In [10]:
structured_llm = model.with_structured_output(Book, method="format_instructions")

structured_llm.invoke("Найди '1984' Оруэлла в английском оригинале")

Book(title='1984', author='George Orwell', language='English')

###  Описание вывода с помощью TypedDict

Если вам не нужно явно валидировать ответ модели, вы можете описать схему вывода с помощью класса TypedDict.

При этом можно использовать специальный синтаксис `Annotated[...]`, который поддерживает LangChain.
С его помощью вы можете задать описания полей, а также указать значения по умолчанию.

> [!NOTE]
> Значения по умолчанию используются только при определении схемы, которая передается в модель.
> Если модель не сгенерирует значение по умолчанию, оно не будет заполнено автоматически.

In [ ]:
from typing import Optional

from typing_extensions import Annotated, TypedDict


# TypedDict
class Book(TypedDict):
    """Книга, которая нужна пользователю"""
    title: Annotated[str, ..., "Название книги"]

    # Также можно указать title следующим образом:

    # title: str                    # без значений по умолчанию и описания
    # title: Annotated[str, ...]    # без значений по умолчанию и описания
    # title: Annotated[str, "foo"]  # без значением по умолчанию, но без описания
    
    author: Annotated[Optional[str], "неизвестен", "Автор книги"]
    language: Annotated[Optional[str], "любой", "Язык написания книги"]

structured_llm = model.with_structured_output(Book)

structured_llm.invoke("Ищу 'Властелин колец' Толкина на английском")

{'author': 'Толкин', 'language': 'английский', 'title': 'Властелин колец'}

### Описание вывода с помощью JSON Schema

Кроме Pydantic- и TypedDict-классов, для описания требуемого вывода модели можно использовать JSON Schema.
Такой подход не требует дополнительного импорта или классов, и наглядно показывает как документируются параметры.

In [ ]:
json_schema = {
          "name": "book",
          "description": "Книга, которая нужна пользователю",
          "parameters": {
              "properties": {
                  "title": {
                      "description": "Название книги",
                      "type": "string"
                  },
                  "author": {
                      "description": "Автор книги",
                      "type": "string",
                      "deafault": "неизвестен"
                  },
                  "language": {
                      "description": "Язык написания книги",
                      "type": "string"
                  }
              },
              "required": [
                  "title",
              ],
              "type": "object"
          },
      }
structured_llm = model.with_structured_output(json_schema)

structured_llm.invoke("Нужна 'Краткая история времени' Хокинга на русском")

{'author': 'Стивен Хокинг',
 'language': 'русский',
 'title': 'Краткая история времени'}

### Добавление примеров (few-shot)

При работе со сложными схемами передайте в модель несколько примеров (*few-shots*) того, каким должен быть ожидаемый вывод.
Примеры можно передать в системном промпте или в списке вызовов инструментов (функций) `tool_calls[]`.


#### Примеры в системном промпте

Примеры запросов пользователя и ожидаемого структурированного вывода можно передать в системном промпте.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system = """Верни из запроса пользователя информацию о книге, которую он хочет найти. Ответ должен содержать title, author и language. 

example_user: Мне нужна книга 'Война и мир'
example_assistant: {{"title": "Война и мир", "author": "неизвестен", "language": "любой"}}

example_user: Нужна сказка 'Маленький принц' Антуана де Сент-Экзюпери на французском.
example_assistant: {{"title": "Маленький принц", "author": "Антуан де Сент-Экзюпери", "language": "французский"}}

example_user: Хочу прочитать 'Сто лет одиночества' Габриэля Гарсиа Маркеса
example_assistant: {{"title": "Сто лет одиночества", "author": "Габриэль Гарсиа Маркес", "language": "любой"}}"""

prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{input}")])

few_shot_structured_llm = prompt | structured_llm
few_shot_structured_llm.invoke("Нужна 'Тень ветра' Карлоса Руиса Сафона на испанском")

{'author': 'Карлос Руис Сафон', 'language': 'испанский', 'title': 'Тень ветра'}

#### Примеры как вызовы инструмента (функции)

Если для структурирования вывода используется вызов инструментов (функций), вы можете передать примеры вызовов в списке `tool_calls[]`.

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage

examples = [
    HumanMessage("Мне нужна книга 'Война и мир'", name="example_user"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {
                "name": "book",
                "args": {
                            "title": "Война и мир",
                            "author": "неизвестен",
                            "language": "любой"
                        },
                "id": "1",
            }
        ],
    ),
    # Большинство моделей, поддерживающих работу с инструментами (функциями), ожидают получение сообщений типа ToolMessage после сообщения типа AIMessage с вызовами инструментов.
    ToolMessage("", tool_call_id="1"),
    HumanMessage("Нужна сказка 'Маленький принц' Антуана де Сент-Экзюпери на французском.", name="example_user"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {
                "name": "joke",
                "args": {
                    "title": "Маленький принц",
                    "author": "Антуан де Сент-Экзюпери",
                    "language": "французский"
                },
                "id": "2",
            }
        ],
    ),
    ToolMessage("", tool_call_id="2"),
    HumanMessage("Хочу прочитать 'Сто лет одиночества' Габриэля Гарсиа Маркеса", name="example_user"),
    AIMessage(
        "",
        tool_calls=[
            {
                "name": "joke",
                "args": {
                    "title": "Сто лет одиночества",
                    "author": "Габриэль Гарсиа Маркес",
                    "language": "любой"
                },
                "id": "3",
            }
        ],
    ),
    ToolMessage("", tool_call_id="3"),
]
system = ""

prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("placeholder", "{examples}"), ("human", "{input}")]
)
few_shot_structured_llm = prompt | structured_llm
few_shot_structured_llm.invoke({"input": "Нужен 'Евгений Онегин' Пушкина в оригинале на русском", "examples": examples})

{'author': 'Пушкин', 'language': 'русский', 'title': 'Евгений Онегин'}

### Получение необработанного вывода

Большие языковые модели не всегда качественно генерируют структурированные данные.
Это может быть особенно заметно при работе со сложными схемами.

Чтобы избежать возможных проблем, вы можете самостоятельно разбирать необработанный ответ модели с помощью собственного или встроенного парсера LangChain.

Для получения необработанного вывода передайте параметр `include_raw=True` при вызове метода `.with_structured_output()`.
В этом случае метод вернет необработанный вывод, результат парсинга и все возникшие ошибки.

In [ ]:
structured_llm = model.with_structured_output(Book, include_raw=True)

structured_llm.invoke("Нужен 'Фауст' Гёте в переводе на русский")

{'raw': AIMessage(content='', additional_kwargs={'function_call': {'name': 'Book', 'arguments': {'author': 'Иоганн Вольфганг Гёте', 'language': 'русский', 'title': 'Фауст'}}, 'functions_state_id': '7640eb48-de06-4794-b91c-895ab69ea8f4'}, response_metadata={'token_usage': {'prompt_tokens': 63, 'completion_tokens': 43, 'total_tokens': 106, 'precached_prompt_tokens': 48}, 'model_name': 'GigaChat-2-Max:2.0.28.02', 'x_headers': {'x-request-id': 'a455038c-c222-45ce-b849-3fe727181a72', 'x-session-id': '1ce13963-0697-4b7c-9cc8-6d6d8bb99e2a', 'x-client-id': None}, 'finish_reason': 'function_call'}, id='a455038c-c222-45ce-b849-3fe727181a72', tool_calls=[{'name': 'Book', 'args': {'author': 'Иоганн Вольфганг Гёте', 'language': 'русский', 'title': 'Фауст'}, 'id': '99e5d0c9-b24a-40c1-9df0-d56ab838507b', 'type': 'tool_call'}]),
 'parsed': {'author': 'Иоганн Вольфганг Гёте',
  'language': 'русский',
  'title': 'Фауст'},
 'parsing_error': None}

## С помощью парсинга вывода модели

При работе со сложными схемами данных модели не всегда могут возвращать удовлетворительный ответ.

Для предотвращения таких ситуаций, вы можете передать в системном промпте инструкции о том, какой формат ответа требуется от модели, после чего самостоятельно разобрать ответ модели.
Для разбора ответа и извлечения структурированных данных из необработанного ответа модели вы можете использовать собственный или встроенный парсер LangChain.

### Встроенный парсер `PydanticOutputParser`

Для разбора ответа, сгенерированного моделью в соответствии с переданной схемой Pydantic, можно использовать парсер `PydanticOutputParser`, встроенный в LangChain.

В пример ниже `format_instructions` добавляется в промпт из метода парсера.


In [ ]:
from typing import List

from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field


class Person(BaseModel):
    """Информация о человеке."""

    name: str = Field(..., description="Имя человека")
    height_in_meters: float = Field(
        ..., description="Рост человека, выраженный в метрах."
    )


class People(BaseModel):
    """Информация обо всех людях в тексте."""

    people: List[Person]


# Подготовка парсера
parser = PydanticOutputParser(pydantic_object=People)

# Промпт
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Wrap the output in `json` tags\n{format_instructions}",
        ),
        ("human", "{query}"),
    ]
).partial(format_instructions=parser.get_format_instructions())

Демонстрация данных, которые передаются в модель.


In [ ]:
query = "Анне 23 года, ее рост 6 футов."

print(prompt.invoke({"query": query}).to_string())

System: Answer the user query. Wrap the output in `json` tags
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"Person": {"description": "Информация о человеке.", "properties": {"name": {"description": "Имя человека", "title": "Name", "type": "string"}, "height_in_meters": {"description": "Рост человека, выраженный в метрах.", "title": "Height In Meters", "type": "number"}}, "required": ["name", "height_in_meters"], "title": "Person", "type": "object"}}, "description": "Информация обо всех людях в тексте.", "properties": {"people": {"items": {"$ref": "#/$defs/Person"}, "title": "P

In [ ]:
chain = prompt | model | parser

chain.invoke({"query": query})

People(people=[Person(name='Анна', height_in_meters=1.83)])

### Собственный парсер

Для разбора ответа модели вы также можете использовать собственный парсер.

In [ ]:
import json
import re
from typing import List

from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field


class Person(BaseModel):
    """Информация о человеке."""

    name: str = Field(..., description="Имя человека")
    height_in_meters: float = Field(
        ..., description="Рост человека, выраженный в метрах."
    )


class People(BaseModel):
    """Информация обо всех людях в тексте."""

    people: List[Person]


# Промпт
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Ответьте на запрос пользователя. Выведите свой ответ в формате JSON, который "
            "соответствует заданной схеме: \`\`\`json\n{schema}\n\`\`\`. "
            "Обязательно заключите ответ в теги \`\`\`json и \`\`\`"
        ),
        ("human", "{query}"),
    ]
).partial(schema=People.schema())


# Собственный парсер
def extract_json(message: AIMessage) -> List[dict]:
    """Extracts JSON content from a string where JSON is embedded between \`\`\`json and \`\`\` tags.

    Parameters:
        text (str): The text containing the JSON content.

    Returns:
        list: A list of extracted JSON strings.
    """
    text = message.content
    # Регулярное выражение для обнаружения блоков JSON.
    pattern = r"\`\`\`json(.*?)\`\`\`"

    # Обнаружение всех неперекрывающихся совпадения регулярного выражения в строке.
    matches = re.findall(pattern, text, re.DOTALL)

    # Вернуть список обнаруженных строк JSON, после удаления любых начальных или конечных пробелов.
    try:
        return [json.loads(match.strip()) for match in matches]
    except Exception:
        raise ValueError(f"Failed to parse: {message}")

<>:31: SyntaxWarning: invalid escape sequence '\`'
<>:32: SyntaxWarning: invalid escape sequence '\`'
<>:41: SyntaxWarning: invalid escape sequence '\`'
<>:31: SyntaxWarning: invalid escape sequence '\`'
<>:32: SyntaxWarning: invalid escape sequence '\`'
<>:41: SyntaxWarning: invalid escape sequence '\`'
C:\Users\Anastasia\AppData\Local\Temp\ipykernel_19248\3397104921.py:31: SyntaxWarning: invalid escape sequence '\`'
  "соответствует заданной схеме: \`\`\`json\n{schema}\n\`\`\`. "
C:\Users\Anastasia\AppData\Local\Temp\ipykernel_19248\3397104921.py:32: SyntaxWarning: invalid escape sequence '\`'
  "Обязательно заключите ответ в теги \`\`\`json и \`\`\`"
C:\Users\Anastasia\AppData\Local\Temp\ipykernel_19248\3397104921.py:41: SyntaxWarning: invalid escape sequence '\`'
  """Extracts JSON content from a string where JSON is embedded between \`\`\`json and \`\`\` tags.
C:\Users\Anastasia\AppData\Local\Temp\ipykernel_19248\3397104921.py:36: PydanticDeprecatedSince20: The `schema` method is 

In [ ]:
query = "Анне 23 года, ее рост 6 футов, а Артему 24 и его рост 6.3 фута."

print(prompt.format_prompt(query=query).to_string())

System: Ответьте на запрос пользователя. Выведите свой ответ в формате JSON, который соответствует заданной схеме: \`\`\`json
{'$defs': {'Person': {'description': 'Информация о человеке.', 'properties': {'name': {'description': 'Имя человека', 'title': 'Name', 'type': 'string'}, 'height_in_meters': {'description': 'Рост человека, выраженный в метрах.', 'title': 'Height In Meters', 'type': 'number'}}, 'required': ['name', 'height_in_meters'], 'title': 'Person', 'type': 'object'}}, 'description': 'Информация обо всех людях в тексте.', 'properties': {'people': {'items': {'$ref': '#/$defs/Person'}, 'title': 'People', 'type': 'array'}}, 'required': ['people'], 'title': 'People', 'type': 'object'}
\`\`\`. Обязательно заключите ответ в теги \`\`\`json и \`\`\`
Human: Анне 23 года, ее рост 6 футов, а Артему 24 и его рост 6.3 фута.


In [ ]:
chain = prompt | model | extract_json

chain.invoke({"query": query})

<unknown>:2: SyntaxWarning: invalid escape sequence '\`'


[{'people': [{'name': 'Анна', 'height_in_meters': 1.83},
   {'name': 'Артем', 'height_in_meters': 1.92}]}]